### LAB 5 [WORKING COPY]

In [1]:
import sklearn
import matplotlib.pyplot as plt
from typing import Tuple
from typing import List

%matplotlib inline

import numpy as np
import pandas as pd
import scipy
import pandas as pd 
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report, precision_recall_curve, average_precision_score
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# df_2017 = pd.read_csv("data/2017.csv.gz",low_memory=False, encoding = "ISO-8859-1", nrows=100000, header=None)

df_2017 = pd.read_csv("data/2017 TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2016 = pd.read_csv("data/2016 TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2015 = pd.read_csv("data/2015 TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2014 = pd.read_csv("data/2014 TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)


df_2017.head()
df_2017.reset_index()
df_2014.head(10)

,0,1,2,3,4,5,6,7
0,ASN00015643,20140101,TMIN,227,NaN,NaN,a,NaN
1,ASN00085296,20140101,TMIN,131,NaN,NaN,a,NaN
2,ASN00085280,20140101,TMIN,84,NaN,NaN,a,NaN
3,CA005030984,20140101,TMIN,-315,NaN,NaN,C,NaN
4,CA003076680,20140101,TMIN,-207,NaN,NaN,C,NaN
5,CA003072151,20140101,TMIN,-258,NaN,NaN,C,NaN
6,CA003031094,20140101,TMIN,-96,NaN,NaN,C,NaN
7,ASN00068151,20140101,TMIN,172,NaN,NaN,a,NaN
8,ARM00087582,20140101,TMIN,228,NaN,NaN,S,NaN
9,USW00024061,20140101,TMIN,-93,NaN,NaN,W,NaN


In [3]:
from functools import reduce
def df_merge_tmin_years(df1, df2):
   # return pd.merge(df1, df2, df3, df4, on=['ID', 'date_mm_dd'], how='outer')
    return pd.merge(df1, df2, how='outer', on=['ID', 'date_mm_dd'])
   # return reduce(lambda left,right: pd.merge(left,right,on=['ID', 'date_mm_dd'], how='outer'), dfs)

In [4]:
import re
def split_it(date):
    return re.findall('(\d\d\d\d)$', date)[0]

def create_date_mm_dd(date):
    return date.apply(split_it)

In [5]:
def rename_columns(df):
    df.rename(index=int,columns = {0:'ID',1:'date', 2:'element',3:'data_value',4:'mFlag',5:'qFlag',6:'sFlag',7:'obsTime'}, inplace=True)

In [6]:
df_2018_test = pd.read_csv("data/2018_test.csv", low_memory=False, encoding = "ISO-8859-1")
df_2018_test.head()
df_2018_test['date_mm_dd'] = create_date_mm_dd(df_2018_test['DATE'].astype('str'))
df_2018_test.head()

,ID,DATE,date_mm_dd
0,ASN00015643,20180101,0101
1,ASN00085296,20180101,0101
2,ASN00085280,20180101,0101
3,CA005030984,20180101,0101
4,CA003076680,20180101,0101


In [7]:
rename_columns(df_2017)
rename_columns(df_2016)
rename_columns(df_2015)
rename_columns(df_2014)


In [8]:
def merge(df):
    return df[['ID','date']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)
df_2017['merged'] = merge(df_2017)
df_2016['merged'] = merge(df_2016)
df_2015['merged'] = merge(df_2015)
df_2014['merged'] = merge(df_2014)
print(df_2014.head())

            ID      date element  data_value mFlag qFlag sFlag  obsTime  \
0  ASN00015643  20140101    TMIN         227   NaN   NaN     a      NaN   
1  ASN00085296  20140101    TMIN         131   NaN   NaN     a      NaN   
2  ASN00085280  20140101    TMIN          84   NaN   NaN     a      NaN   
3  CA005030984  20140101    TMIN        -315   NaN   NaN     C      NaN   
4  CA003076680  20140101    TMIN        -207   NaN   NaN     C      NaN   

                merged  
0  ASN0001564320140101  
1  ASN0008529620140101  
2  ASN0008528020140101  
3  CA00503098420140101  
4  CA00307668020140101  


In [9]:
df_2014.columns

Index(['ID', 'date', 'element', 'data_value', 'mFlag', 'qFlag', 'sFlag',
       'obsTime', 'merged'],
      dtype='object')

In [10]:
print(len(df_2014))

672650


In [11]:
def pivot(df):
    return df.pivot_table( index=['merged','date', 'ID'], columns='element', values='data_value')
df_2017_pivoted = pivot(df_2017)
df_2016_pivoted = pivot(df_2016)
df_2015_pivoted = pivot(df_2015)
df_2014_pivoted = pivot(df_2014)

df_2017_pivoted.reset_index(inplace=True)
df_2016_pivoted.reset_index(inplace=True)
df_2015_pivoted.reset_index(inplace=True)
df_2014_pivoted.reset_index(inplace=True)

# df = df.pivot_table(
#     values='value',
#     index=['stream_name', 'preferred_timestamp', 'internal_timestamp'],
#     columns='value_id'
#     )

In [12]:
df_2017_pivoted.head()

element,merged,date,ID,TMIN
0,AE00004119620170103,20170103,AE000041196,156
1,AE00004119620170105,20170105,AE000041196,176
2,AE00004119620170107,20170107,AE000041196,130
3,AE00004119620170108,20170108,AE000041196,128
4,AE00004119620170109,20170109,AE000041196,126


In [13]:
print(df_2017_pivoted.head())
print(df_2016_pivoted.head())
print(df_2015_pivoted.head())
print(df_2014_pivoted.head())


# # Tmin( T-10 to T-1) Location LocationId
# Training

# T-10        T-9           T-8         T-1          -------  T-0 (y)
# Tmin20171231 Tmin20171230 Tmin20171229 Tmin20171228 ---- Tmin201712 (y)Tmin20181231
# Tmin20171230 Tmin20171228 Tmin20171227
# Test

# Tmin20171231 Tmin20171230 Tmin20171229 Tmin20171228 ---- Tmin201712 (y)Tmin20180101



# 0101 -> 0107 (Average of the previous week)


# Run the model for first 10 days
# Get y values for those
# Substitute those values as features for the next 10 days
# repeat for 50 days (5 times)


element               merged      date           ID  TMIN
0        AE00004119620170103  20170103  AE000041196   156
1        AE00004119620170105  20170105  AE000041196   176
2        AE00004119620170107  20170107  AE000041196   130
3        AE00004119620170108  20170108  AE000041196   128
4        AE00004119620170109  20170109  AE000041196   126
element               merged      date           ID  TMIN
0        AE00004119620160101  20160101  AE000041196   155
1        AE00004119620160102  20160102  AE000041196   188
2        AE00004119620160105  20160105  AE000041196   139
3        AE00004119620160108  20160108  AE000041196   147
4        AE00004119620160109  20160109  AE000041196   136
element               merged      date           ID  TMIN
0        AE00004119620150101  20150101  AE000041196   125
1        AE00004119620150102  20150102  AE000041196   127
2        AE00004119620150103  20150103  AE000041196   140
3        AE00004119620150107  20150107  AE000041196   136
4        AE000

In [14]:
df_2017_pivoted['date_mm_dd'] = create_date_mm_dd(df_2017_pivoted['date'].astype('str'))
df_2016_pivoted['date_mm_dd'] = create_date_mm_dd(df_2016_pivoted['date'].astype('str'))
df_2015_pivoted['date_mm_dd'] = create_date_mm_dd(df_2015_pivoted['date'].astype('str'))
df_2014_pivoted['date_mm_dd'] = create_date_mm_dd(df_2014_pivoted['date'].astype('str'))


In [15]:
len(df_2017_pivoted)

634291

In [16]:
df_2017_pivoted.columns

Index(['merged', 'date', 'ID', 'TMIN', 'date_mm_dd'], dtype='object', name='element')

In [17]:
def tmin_rename(df, year):
    df.rename(index=str ,columns = {'TMIN':'TMIN_%s'% year}, inplace=True)

tmin_rename(df_2014_pivoted, '2014')
tmin_rename(df_2015_pivoted, '2015')
tmin_rename(df_2016_pivoted, '2016')
tmin_rename(df_2017_pivoted, '2017')

In [18]:
df_compiled_2017_2016 = df_merge_tmin_years(df_2017_pivoted, df_2016_pivoted)
df_compiled_2017_2016_2015 = df_merge_tmin_years(df_compiled_2017_2016, df_2015_pivoted)
df_compiled_2017_2016_2015_2014 = df_merge_tmin_years(df_compiled_2017_2016_2015, df_2014_pivoted)


In [19]:
df_compiled_2017_2016_2015_2014.head()


element,merged_x,date_x,ID,TMIN_2017,date_mm_dd,merged_y,date_y,TMIN_2016,merged_x,date_x,TMIN_2015,merged_y,date_y,TMIN_2014
0,AE00004119620170103,20170103.0,AE000041196,156.0,0103,NaN,NaN,NaN,AE00004119620150103,20150103.0,140.0,AE00004119620140103,20140103.0,140.0
1,AE00004119620170105,20170105.0,AE000041196,176.0,0105,AE00004119620160105,20160105.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AE00004119620170107,20170107.0,AE000041196,130.0,0107,NaN,NaN,NaN,AE00004119620150107,20150107.0,136.0,NaN,NaN,NaN
3,AE00004119620170108,20170108.0,AE000041196,128.0,0108,AE00004119620160108,20160108.0,147.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AE00004119620170109,20170109.0,AE000041196,126.0,0109,AE00004119620160109,20160109.0,136.0,NaN,NaN,NaN,AE00004119620140109,20140109.0,115.0


In [20]:
df_clean = df_compiled_2017_2016_2015_2014[['date_mm_dd', 'ID', 'TMIN_2014', 'TMIN_2015', 'TMIN_2016', 'TMIN_2017' ]]

In [21]:
print(df_clean.isnull().sum())
print(len(df_clean))
df_clean.head()

element
date_mm_dd         0
ID                 0
TMIN_2014      97077
TMIN_2015     104439
TMIN_2016     117329
TMIN_2017     135436
dtype: int64
769727


element,date_mm_dd,ID,TMIN_2014,TMIN_2015,TMIN_2016,TMIN_2017
0,0103,AE000041196,140.0,140.0,NaN,156.0
1,0105,AE000041196,NaN,NaN,139.0,176.0
2,0107,AE000041196,NaN,136.0,NaN,130.0
3,0108,AE000041196,NaN,NaN,147.0,128.0
4,0109,AE000041196,115.0,NaN,136.0,126.0


In [22]:
def get_avg_2018(df):
    return df.mean(axis = 1, skipna=True)
    #return df

df_clean['mean'] = get_avg_2018(df_clean[['TMIN_2014', 'TMIN_2015', 'TMIN_2016', 'TMIN_2017' ]])



C:\Users\mhamsho\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [23]:
print(len(df_clean))
df_clean.head()

769727


element,date_mm_dd,ID,TMIN_2014,TMIN_2015,TMIN_2016,TMIN_2017,mean
0,0103,AE000041196,140.0,140.0,NaN,156.0,145.333333
1,0105,AE000041196,NaN,NaN,139.0,176.0,157.500000
2,0107,AE000041196,NaN,136.0,NaN,130.0,133.000000
3,0108,AE000041196,NaN,NaN,147.0,128.0,137.500000
4,0109,AE000041196,115.0,NaN,136.0,126.0,125.666667


In [24]:
df_clean.isnull().sum()

element
date_mm_dd         0
ID                 0
TMIN_2014      97077
TMIN_2015     104439
TMIN_2016     117329
TMIN_2017     135436
mean               0
dtype: int64

In [25]:
# Combine 2018 test data with average
df_pred = pd.merge(df_2018_test, df_clean, how='left', on=['ID', 'date_mm_dd'])
df_pred[df_pred['mean'].isnull()]


,ID,DATE,date_mm_dd,TMIN_2014,TMIN_2015,TMIN_2016,TMIN_2017,mean
58,RSM00034535,20180101,0101,NaN,NaN,NaN,NaN,NaN
142,FP000091948,20180101,0101,NaN,NaN,NaN,NaN,NaN
288,KZ000035849,20180101,0101,NaN,NaN,NaN,NaN,NaN
289,IRM00040858,20180101,0101,NaN,NaN,NaN,NaN,NaN
354,MD000033881,20180101,0101,NaN,NaN,NaN,NaN,NaN
357,IN021010100,20180101,0101,NaN,NaN,NaN,NaN,NaN
435,RSM00030695,20180101,0101,NaN,NaN,NaN,NaN,NaN
440,MOM00060220,20180101,0101,NaN,NaN,NaN,NaN,NaN
446,IT000016560,20180101,0101,NaN,NaN,NaN,NaN,NaN
508,RSM00034958,20180101,0101,NaN,NaN,NaN,NaN,NaN


In [26]:
df_pred_formatted = df_pred[['ID', 'DATE', 'mean']]
df_pred_formatted['mean'].fillna(0, inplace=True)
df_pred_formatted['SUB_ID'] = df_pred_formatted[['DATE','ID']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)


C:\Users\mhamsho\Anaconda\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\mhamsho\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
df_final = df_pred_formatted[['SUB_ID', 'mean']]

In [28]:
df_final.tail()

,SUB_ID,mean
397799,20180220USC00304174,-100.0
397800,20180220USC00246647,-44.5
397801,20180220USC00094170,25.0
397802,20180220ROE00108889,12.0
397803,20180220USC00086092,116.5


In [29]:

df_final.to_csv('dma_submission_1.csv')

In [30]:
df_final.head()

,SUB_ID,mean
0,20180101ASN00015643,213.25
1,20180101ASN00085296,152.50
2,20180101ASN00085280,121.75
3,20180101CA005030984,-212.50
4,20180101CA003076680,-164.75


In [31]:
df_clean[df_clean.ID == 'AE000041196'].mean()

element
date_mm_dd    2.102255e+192
TMIN_2014      1.332812e+02
TMIN_2015      1.406957e+02
TMIN_2016      1.360526e+02
TMIN_2017      1.440000e+02
mean           1.385612e+02
dtype: float64

## Marc's addition

In [32]:
## WARNING ##
# This cell takes a long time to run!!! (about 45min on my mac book pro)
# It simply calculates the average per station for all January / Februray over the 4 years of history
# This average per station is the predicted value

# for station in df_clean.ID.unique():
#     station_winter_average = df_clean[df_clean.ID == station].mean()[0:4].mean()
#     df_clean.loc[df_clean.ID == station, 'station_winter_average'] = station_winter_average

In [33]:
# Combine 2018 test data with average
# df_pred2 = pd.merge(df_2018_test, df_clean, how='left', on=['ID', 'date_mm_dd'])

# df_pred_formatted2 = df_pred2[['ID', 'DATE', 'station_winter_average']]
# df_pred_formatted2['station_winter_average'].fillna(0, inplace=True)
# df_pred_formatted2['SUB_ID'] = df_pred_formatted2[['DATE','ID']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

# df_final2 = df_pred_formatted2[['SUB_ID', 'station_winter_average']]

# df_final2 = pd.DataFrame(np.insert(df_final2.values, 0, values=['SUB_ID', 'DATA_VALUE'], axis=0))


In [34]:
# print(len(df_final2))
# df_final2.head()


In [35]:
# df_final2.to_csv('dma_submission_2.csv', index = False, header = False)

 --- That's it for Marc ----
  
   
    
     
      

-- That's it for Nasha --

In [36]:
#ideas: Cluster based on id and tmin and date
# Find previous based on previous week's data only for the first 7 days- use the rest as they are
# Featurise based on location, data and id

# training data

# 1 Location(State) LocationID Tmin (T-10) Tmin (T-9) ---(T-1)   y-> T0
# 2 

In [37]:
#print(df_clean.tail())
df_clean['TMIN_2014'].fillna(method='ffill', inplace=True)
df_clean['TMIN_2015'].fillna(method='ffill', inplace=True)
df_clean['TMIN_2016'].fillna(method='ffill', inplace=True)
df_clean['TMIN_2017'].fillna(method='ffill', inplace=True)
df_clean['mean'] = get_avg_2018(df_clean[['TMIN_2014', 'TMIN_2015', 'TMIN_2016', 'TMIN_2017' ]])
df_clean['TMIN_2016'].iloc[0] = 140


C:\Users\mhamsho\Anaconda\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\mhamsho\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mhamsho\Anaconda\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Us

In [38]:

df_clean.tail()

element,date_mm_dd,ID,TMIN_2014,TMIN_2015,TMIN_2016,TMIN_2017,mean
769722,0214,ZA000067743,190.0,124.0,194.0,186.0,173.50
769723,0215,ZA000067743,205.0,124.0,194.0,186.0,177.25
769724,0217,ZA000067743,175.0,124.0,194.0,186.0,169.75
769725,0219,ZA000067743,185.0,124.0,194.0,186.0,172.25
769726,0220,ZA000067743,160.0,124.0,194.0,186.0,166.00


In [39]:
import re
def split_state(value):
    x = re.match('[^0]*', value)
    return x.group(0)

def create_state(col):
    return col.apply(split_state)

df_clean['state'] = create_state(df_clean['ID'].astype('str'))
df_2018_test['state'] = create_state(df_2018_test['ID'].astype('str'))

C:\Users\mhamsho\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
x = list(df_clean.state.unique())

In [44]:
y = list(df_2018_test.state.unique())

In [45]:
state_intersection = [i for i , j in zip(x,y)]

In [46]:
df_clean_intersection =df_clean[df_clean['state'].isin(state_intersection)]

In [47]:
from sklearn.feature_extraction import DictVectorizer #to turn categorial variables into numeric arrays

X = df_clean_intersection[[ 'state', 'date_mm_dd']]
X_dict = X.to_dict( orient = 'records' )
vec = DictVectorizer()
X_final = vec.fit_transform(X_dict).toarray()



In [48]:
X_test = df_2018_test[[ 'state', 'date_mm_dd']]
X_dict = X_test.to_dict( orient = 'records' )
vec = DictVectorizer()
X_final_test = vec.fit_transform(X_dict).toarray()

### IGNORE THIS PART: MOUDI's work

In [ ]:
df_clean_2 = df_clean.drop(['TMIN_2014', 'TMIN_2015', 'TMIN_2016','TMIN_2017', 'state'], axis=1)

In [ ]:
df_clean_2.head()

In [ ]:
df_pred_2 = pd.merge(df_2018_test, df_clean_2, how='left', on=['ID', 'date_mm_dd'])


In [ ]:
df_pred_2.drop(['state', 'date_mm_dd'],axis=1, inplace=True)

In [ ]:
df_pred_2.head()

In [ ]:
df_2018_test.head()

In [ ]:
df_pred_2['SUB_ID'] = df_pred_2[['DATE','ID']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

In [ ]:
df_final_2 = df_pred_2[['SUB_ID','mean']]

In [ ]:
df_final_2.isnull().sum()

In [ ]:
df_final_2['mean'].fillna(method='ffill', inplace=True)


In [ ]:
df_final_2.isnull().sum()

In [ ]:
len(df_final_2)

In [ ]:
len(df_2018_test)

In [ ]:
df_final_2.reset_index(drop=True, inplace=True)

In [ ]:
df_final_2.rename(columns={'mean':'DATA_VALUE'},inplace=True)

In [ ]:
df_final_2.to_csv('submission_4.csv',index=False)

#### END

## MODEL BUILDING START

In [ ]:
df_2018_test.head()

In [50]:
len(X_final)

761958

In [51]:
len(y)

769727

In [53]:
#models
#X = df_clean[['ID', 'state', 'date_mm_dd']]
y = df_clean_intersection['mean']

# X_test = df_2018_test[['ID', 'state', 'date_mm_dd']]
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg = linreg.fit(X_final, y)

print(linreg.score(X_final,y))

y_pred = linreg.predict(X_final)

# print("Mean squared error: %.2f"
#      % mean_squared_error(y_test, y_pred))

#print(linreg.score(X_test,y_test))
print(y_pred)

0.526165902018
[ 140.90570068  124.29168701  120.93780518 ...,  243.06567383  248.32489014
  251.25372314]


In [54]:
print('Accuracy of Linear Regression on test set: {:.5f}'.format(linreg.score(X_final, y)))

Accuracy of Linear Regression on test set: 0.52617


### 2018_test run

In [55]:
y_pred_2018 = linreg.predict(X_final_test)

In [58]:
len(y_pred_2018)

397804

In [57]:
df_2018_test.shape

(397804, 4)

In [59]:
df_2018_test['DATA_VALUE'] = y_pred_2018

In [62]:
df_final['mean'] = y_pred_2018

C:\Users\mhamsho\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
df_final.rename(columns={'mean':'DATA_VALUE'}, inplace=True)

C:\Users\mhamsho\Anaconda\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [64]:
df_final.head()

,SUB_ID,DATA_VALUE
0,20180101ASN00015643,181.947937
1,20180101ASN00085296,181.947937
2,20180101ASN00085280,181.947937
3,20180101CA005030984,-121.853333
4,20180101CA003076680,-121.853333


In [65]:
df_final.to_csv('submission_5.csv',index=False)

#### MODEL FINISH

In [ ]:
print(X_final.shape)

In [ ]:
df_clean.tail()

df_clean['location'], df_clean['location_id'] = df_clean['ID'].str.split('0', 1, expand=True)
df_clean.head()

In [ ]:
submission_sample = pd.read_csv("data/sample_submission.csv", low_memory=False, encoding = "ISO-8859-1", header=None)


In [ ]:
print(len(submission_sample))
submission_sample.head()

In [ ]:
df_unstacked_core = df_unstacked[[ 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN']]

In [ ]:
df_unstacked_core.head()

In [ ]:
df_unstacked_core.isnull().sum()

In [ ]:
len(df_unstacked_core)

#### Mahmoud 

In [ ]:
df_unstacked['TMIN'].isnull().sum()

In [ ]:
df_unstacked['TMAX'].isnull().sum()

In [ ]:
df_unstacked.shape

In [ ]:
df_2017.shape

In [ ]:
df_unstacked['TMIN'].describe()

In [ ]:
temp_df = df_unstacked.query(('TMAX >= 0 or TMAX <0') or ('TMIN >=0 or TMIN <0'))

In [ ]:
temp_df['TMIN'] = temp_df['TMIN'].fillna(temp_df['TMIN'].mean())

In [ ]:
temp_df = pd.merge(temp_df, df_2017_filtered, right_index=True, left_index=True)

In [ ]:
temp_df.columns

In [ ]:
temp_df.columns
columns = ['AWDR', 'AWND', 'DAPR', 'DATN', 'DWPR', 'EVAP', 'MDPR', 'MDTN', 'MNPN', 'MXPN', 'PGTM', 'PRCP', 'PSUN', 'SN31', 'SN32', 'SN33',
       'SN35', 'SN36', 'SN51', 'SN52', 'SN53', 'SN55', 'SN56', 'SNOW', 'SNWD',
       'SX31', 'SX32', 'SX33', 'SX35', 'SX36', 'SX51', 'SX52', 'SX53', 'SX55',
       'SX56', 'TAVG', 'THIC', 'TMAX', 'TMIN', 'TOBS', 'TSUN', 'WDF2', 'WDF5',
       'WDFG', 'WDMV', 'WESD', 'WESF', 'WSF2', 'WSF5', 'WSFG', 'WSFI', 'WT01',
       'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11', 'obsTime']

In [ ]:
y = temp_df['TMIN']
X = temp_df[columns]

In [ ]:
X.columns

In [ ]:
X.fillna(X.mean(), inplace=True)

In [ ]:
X.shape

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# feature extraction
svc = LinearRegression()
rfe = RFE(estimator=svc, n_features_to_select=10, step=100)
RFE = rfe.fit(X, y)

print(RFE)
print(RFE.n_features_)
print(RFE.support_)
print(RFE.ranking_)

In [ ]:
selected_feature = []
rfe_list = RFE.support_
for index in range(len(rfe_list)):
    if rfe_list[index] == True:
        selected_feature.append(columns[index])
    else:
        continue

In [ ]:
X.drop('TMIN',axis=1,inplace=True)

In [ ]:
X.describe()

In [ ]:
y.shape

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [ ]:
print('Accuracy of Linear Regression on test set: {:.5f}'.format(regr.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score


print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

In [ ]:
import xgboost as xgb

# specify parameters via map
gbm = xgb.XGBClassifier(max_depth=20, learning_rate=0.2).fit(X_train, y_train)
predictions = gbm.predict(X_test)


In [ ]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, predictions))